# Practical test of standardiser coverage on VDJdb data

## Setup

In [1]:
import sys
import os
from pathlib import Path

if not "PROJECT_PATH" in globals():
    PROJECT_PATH = Path.cwd().parent.resolve()

sys.path.append(PROJECT_PATH)
os.chdir(PROJECT_PATH)

In [2]:
import json
import pandas as pd
import re
import tidytcells

In [3]:
df = pd.read_csv(Path("data") / "vdjdb.tsv.gz", sep="\t")

In [4]:
df.head()

,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...",2
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAVPSGAGSYQLTF"", ""cdr3_old"": ""CAVPSG...",2
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2


## Test TCR standardisation

In [5]:
df.apply(
    lambda row: pd.NA
    if pd.isna(row["V"])
    else tidytcells.tcr.standardise(
        row["V"], species=row["Species"].lower(), enforce_functional=True
    ),
    axis=1,
)

/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:25: UserWarning: Unsupported species: "macacamulatta". Skipping TCR gene standardisation procedure...
  warn(


0        TRAV26-1*01
1          TRBV13*01
2          TRBV13*01
3          TRAV20*01
4          TRBV13*01
            ...     
89316    TRBV13-3*01
89317    TRAV14-2*01
89318     TRAV8-2*01
89319     TRAV6-5*01
89320    TRAV9D-4*01
Length: 89321, dtype: object

In [6]:
df.apply(
    lambda row: pd.NA
    if pd.isna(row["J"])
    else tidytcells.tcr.standardise(
        row["J"], species=row["Species"].lower(), enforce_functional=True
    ),
    axis=1,
)

/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:25: UserWarning: Unsupported species: "macacamulatta". Skipping TCR gene standardisation procedure...
  warn(


0         TRAJ43*01
1        TRBJ1-5*01
2        TRBJ1-5*01
3         TRAJ28*01
4        TRBJ1-5*01
            ...    
89316    TRBJ2-3*01
89317     TRAJ23*01
89318     TRAJ30*01
89319     TRAJ52*01
89320     TRAJ31*01
Length: 89321, dtype: object

## Test MHC standardisation

In [7]:
df.apply(
    lambda row: pd.NA
    if pd.isna(row["MHC A"])
    else tidytcells.mhc.standardise(row["MHC A"], species=row["Species"].lower()),
    axis=1,
)

/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "H-2Kb" for species musmusculus. Attempted fix "H-2KB" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "H-2Ld" for species musmusculus. Attempted fix "H-2LD" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "H-2Kd" for species musmusculus. Attempted fix "H-2KD" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "HLA-A*08:01" for species homosapiens. Attempted fix "HLA-A*008:001" did not meet the standardised format requirements. Ignoring this

0        HLA-B*08
1        HLA-B*08
2        HLA-B*08
3        HLA-B*08
4        HLA-B*08
           ...   
89316      MH2-D1
89317      MH2-D1
89318      MH2-D1
89319      MH2-D1
89320      MH2-D1
Length: 89321, dtype: object

In [8]:
def _resolve_alleles(alleles: str) -> list:
    """
    Resolve incomplete MHC gene names from donor.MHC fields in VDJdb, and
    return as a list.
    """
    alleles = re.split(",|;", alleles)
    alleles = [allele.strip() for allele in alleles if allele != ""]

    resolved_alleles = []
    most_recent_gene_base = None

    for allele in alleles:
        current_gene_base = re.match(r"(HLA-)?([ABCEFG]|D[PQR][AB]\d?)", allele)

        if current_gene_base:
            most_recent_gene_base = current_gene_base.group(0)
            resolved_alleles.append(allele)
            continue

        if most_recent_gene_base:
            resolved_alleles.append(f"{most_recent_gene_base}*{allele}")
            continue

        resolved_alleles.append(allele)

    return resolved_alleles


meta_mhcs = df[["Species"]].copy()
meta_mhcs["MHC"] = (
    df["Meta"]
    .transform(lambda x: json.loads(x)["donor.MHC"])
    .transform(lambda x: None if x == "" else x)
)
meta_mhcs = meta_mhcs[meta_mhcs["MHC"].notna()]
meta_mhcs["MHC"] = meta_mhcs["MHC"].map(_resolve_alleles)
meta_mhcs = meta_mhcs.explode("MHC")
meta_mhcs = meta_mhcs.drop_duplicates()

meta_mhcs.apply(
    lambda row: pd.NA
    if pd.isna(row["MHC"])
    else tidytcells.mhc.standardise(row["MHC"], species=row["Species"].lower()),
    axis=1,
).notna().sum() / len(meta_mhcs)

/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "HLA-DRB*07:01" for species homosapiens. Attempted fix "HLA-DRB*007:001" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "HLA-DRB*13:01" for species homosapiens. Attempted fix "HLA-DRB*013:001" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "HLA-DRB*08:03:2" for species homosapiens. Attempted fix "HLA-DRB*008:003:2" did not meet the standardised format requirements. Ignoring this gene name...
  warn(
/home/yutanagano/Projects/tidytcells/src/tidytcells/_utils/warnings.py:14: UserWarning: Failed to standardise: "HLA-DRB*15:01:1" for species homosapiens. Attempted fix "HLA-DRB*015:001:1

0.9232736572890026